In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [2]:
# Read in the appropriate datasets with selected features
players = pd.read_csv("players.csv")[["element", "round", "total_points", "value" ]]
plr_info = pd.read_csv("basic_player_info.csv")[["element_type", "id", "web_name", "short_name"]]
cumulative = pd.read_csv("cumulative_points.csv")[["element", "round", "cum_points", "cost"]]

In [323]:
# We need a data set that holds all player gameweeks including metrics from each gameweek
#Create a function that takes the data and merges into a useful set

def merge_data(players, plr_info, cumulative):
    """
    In this function we'll take the available datasets, filter for useful features and values and merge into one 
    master set
    """

    # join the players and info sets
    data = players.merge(plr_info, how="left", left_on="element", right_on="id").drop("element", axis=1)
    
    # join the data and cumulative points sets
    data = data.merge(cumulative, how="left", left_on=["id", "round"], 
                      right_on=["element", "round"]).drop("element", axis=1)


    data = data[["id", "web_name", "round", "total_points", "element_type", "short_name", "cum_points", "value"]]
    
    return data

In [324]:
data = merge_data(players, plr_info, cumulative)

In [325]:
data.head()

,id,web_name,round,total_points,element_type,short_name,cum_points,value
0,1,Özil,1,0,3,ARS,0.0,70
1,1,Özil,2,0,3,ARS,0.0,69
2,1,Özil,3,0,3,ARS,0.0,69
3,1,Özil,4,0,3,ARS,0.0,68
4,1,Özil,5,0,3,ARS,0.0,68


In [321]:
###

In [340]:
def round_players(full_set, gw_idx):
    
    """
    In this function we'll create a set of available, scoring players for the requested gameweek. From that set 
    we'll sort by points scored and generate a set of four potential transfers, 1 per position and store them 
    in a targets dataset.
    
    We return the available players set and the targets
    
    We need to add in some functionality to check that our targets are not already in the current team
    
    
    """
    
    # First we'll check which players were available at the stated gameweek
    gw_true = full_set[full_set["round"]==gw_idx]
    # Create a list of ids for these players
    gw_id_list = gw_true.id.tolist()
    # Now select all the available players in all rounds from our data
    available_players = full_set.loc[data["id"].isin(gw_id_list)]

    # Now we'll remove all players who have 0 points at the latest gw, first filter for latest gw
    latest_round = available_players[available_players["round"]==gw_idx]
    # Now filter in latest round for players scoring 1 or more points
    scorers = latest_round[latest_round.cum_points>0]

    # Get the gw 2 costs from the cumulative df cost feature
    week_cost = cumulative[cumulative["round"]==gw_idx][["element", "cost"]]

    rd_player_set= scorers.merge(week_cost, how="left", left_on="id", right_on="element").drop("element", axis=1)

    # Finally we create a value feature (points per £million) which will assist our team selection
    dec = 1
    rd_player_set["value"] = rd_player_set.cum_points / rd_player_set.cost
    rd_player_set["value"] = rd_player_set["value"].apply(lambda x: round(x, dec)) # Round the values down to 1 decimal place
    
    #if team == None

    
    
    return rd_player_set

In [362]:
def target_generator(rd_player_set, updated_team):
    
    """
    In this function we will set transfer targets using the following steps
    1. Get the set of players for the upcoming gameweek, sorted in order of ascending points scored
    2. Set the top performer in each position to a transfers dataframe
    3. Check that these transfer targets are not already held in our squad
    4. If we already have a player, we go to the next best scoring player in that position and add him to targets
    5. Return the targets to the managing function
    """
    
    # Now generate a target per position
    gk2 = rd_player_set[rd_player_set.element_type==1].sort_values(by=["total_points", "cost"], ascending=[False, True])
    def2 = rd_player_set[rd_player_set.element_type==2].sort_values(by=["total_points", "cost"], ascending=[False, True])
    mid2 = rd_player_set[rd_player_set.element_type==3].sort_values(by=["total_points", "cost"], ascending=[False, True])
    att2 = rd_player_set[rd_player_set.element_type==4].sort_values(by=["total_points", "cost"], ascending=[False, True])
    
    # Set up an empty dataframe to store the next players in each position
    targets = pd.DataFrame(columns = rd_player_set.columns)

    # Initialise i to 0 although this will change as we iterate 
    #### If the initial 4 players are rejected we'll need to go to the next level down..
    ### We would expect to only go 3 or 4 levels deep in the worst case scenario?? More thinking required here though
    i=0
    next_gk = gk2.iloc[i].to_frame().T
    next_def = def2.iloc[i].to_frame().T
    next_mid = mid2.iloc[i].to_frame().T
    next_att = att2.iloc[i].to_frame().T

    targets = targets.append(next_gk).append(next_def).append(next_mid).append(next_att)
    
    # Now we reset the index so we can iterate and search by index
    targets = targets.reset_index(drop=True)
    
    ## Now we need to scroll through available players until our targets are not already in our current squad
    
    # We do a loop through the 4 targets, 4 times! 
    # This is not a general solution to this problem. We assume that we will remove duplicate players in 
    # targets and current team after four passes through the player pool. 
   
    for i in range(targets.shape[0]):
        for j in range(targets.shape[0]):
            # Have we found a duplicate player / transfer target
            if targets.iloc[j].id in updated_team.id.tolist(): 
                targets = targets.drop(index=j, axis=0)
                if j == 0:
                    # Get the next player in the goalkeepers pool (max. 4 deep)
                    targets = targets.append(gk2.iloc[i+1].to_frame().T) 
                    targets = targets.sort_values(by="element_type")
                    targets = targets.reset_index(drop=True)
                    #print("y")
                elif j == 1:
                    # Get the next player in the defenders pool (max. 4 deep)
                    targets = targets.append(def2.iloc[i+1].to_frame().T) 
                    targets = targets.sort_values(by="element_type")
                    targets = targets.reset_index(drop=True)
                    #print("y")

                elif j == 2:
                    # Get the next player in the midfielder pool (max. 4 deep)
                    targets = targets.append(mid2.iloc[i+1].to_frame().T) 
                    targets = targets.sort_values(by="element_type")
                    targets = targets.reset_index(drop=True)
                    #print("y")


                elif j == 3:
                    # Get the next player in the attackers pool (max. 4 deep)
                    targets = targets.append(att2.iloc[i+1].to_frame().T) 
                    targets = targets.sort_values(by="element_type")
                    targets = targets.reset_index(drop=True)
                    #print("y")
            else:
                #print("n")
                continue

    
    print(targets.web_name, targets.total_points)
    return targets

In [363]:

# Create a function that builds our baseline team - i.e. a team consisting of highest points scorers
def baseline_team(rd_players):
    """
    In this function we'll build a baseline GW1 team that selects the highest scoring players regardless of costs
    """
    # Call the prepset to create the full set of available players
    df = rd_players
    
    # Create position sets sorted in descending order of points scored
    gk = df[df.element_type==1].sort_values(by="cum_points", ascending=False)
    dfn = df[df.element_type==2].sort_values(by="cum_points", ascending=False)
    mid = df[df.element_type==3].sort_values(by="cum_points", ascending=False)
    fwd = df[df.element_type==4].sort_values(by="cum_points", ascending=False)
    
    # Select the players for our squad based on the number of allowed players per position in the FPL game
    gk_selected = gk.iloc[0:2]
    def_selected = dfn.iloc[0:5]
    mid_selected = mid.iloc[0:5]
    fwd_selected = fwd.iloc[0:3]

    # Store the squad player's ids in a dataframe. We use ids over names as some names are duplicated
    gk_ids = gk_selected.id.values.tolist()
    def_ids = def_selected.id.values.tolist()
    mid_ids = mid_selected.id.values.tolist()
    fwd_ids = fwd_selected.id.values.tolist()

    
    # Let's store the team in a data frame
    team_list = [gk_ids, def_ids, mid_ids, fwd_ids]
    team_frame = pd.DataFrame(team_list).T
    team_frame.rename(columns = {0: "gk", 1:"def", 2:"mid", 3:"fwd"}, inplace=True)

    # Now, select the team_frame data points from our prepared df to create a the base team
    base_team = df[df.id.isin(team_frame.values.reshape((20)).tolist())].sort_values(by="element_type", 
                                                                                     ascending=True)
    
    
    # Finally we sort the players by element and then in descending order of points scored
    base_gk = base_team[base_team.element_type==1].sort_values(by="cum_points", ascending=False)
    base_dfn = base_team[base_team.element_type==2].sort_values(by="cum_points", ascending=False)
    base_mid = base_team[base_team.element_type==3].sort_values(by="cum_points", ascending=False)
    base_fwd = base_team[base_team.element_type==4].sort_values(by="cum_points", ascending=False)

    base_team = pd.DataFrame(columns=base_gk.columns)
    base_team = base_team.append(base_gk).append(base_dfn).append(base_mid).append(base_fwd)
    base_team.reset_index(inplace=True)
    base_team.drop("index", axis=1, inplace=True)
    
    # We'll need to access the team cost for budgeting
    team_cost = base_team.cost.sum()/10
    
    return base_team, team_cost

In [364]:
def double_or_missing(current_team_next, gw):
    
    """
    In this function we manage players who have double gameweeks, this is a problem for the code as it searches by 
    gameweek and if a player has two entries for a gameweek this causes errors / inaccurate scoring.
    
    Process as follows:
    1. We get all the entries for current squad for the next gameweek
    2. If any are involved in double gameweeks, we drop the duplicate entries
    3. We then check back in to the master set and tot up the player points in a double gameweek
    4. We reset the points in our squad set to the summed points set in point #3 above
    5. We then return this revised team as the current team points for the upcoming gameweek.
    """
    
    
    ## SOLVE MULTIPLE GAMEWEEKS
    # Do this after the current team next is generated, strip all duplicated players if there is a double gw
    current_team_next = current_team_next.drop_duplicates(["id"])
    
    # Get the unique ids of the stripped team gameweek
    unique_ids= current_team_next.id.unique() 
    
    
    for i, j in enumerate(unique_ids):
        
        # If the player has multiple entries for one gameweek this means he had a double gw
        if data[(data["round"]==gw) & (data.id==j)].shape[0] > 1:
        
            # Get the player data points for that gameweek
            player_multiple_entries = data[(data["round"]==gw) & (data.id==j)]

            # Reduce the number of entries to two, this indicates a double gameweek
            player_reduced_entries = player_multiple_entries.iloc[:2]

            # Sum up the points for the double gameweek
            player_summed_points = player_reduced_entries.total_points.iloc[0] + player_reduced_entries.total_points.iloc[1]

            # Reduce player gameweek to one data point 
            player_single = player_reduced_entries.iloc[1]

            # Set the gw score to the summed points
            player_single["total_points"] = player_summed_points

    
            # Change the total points for that entry to the summed points for the gw
            current_team_next.loc[current_team_next.id==j, "total_points"] = player_summed_points
    
    return current_team_next

In [365]:
#new_team_rev

In [366]:
def team_sorted(round_player_set, current_team): # pass in the existing team
    
    """
    In this function we look at how our team performs in the gameweek and order them so that the worst performers
    inhabit the last index location for each position (playing position). 
    
    We then can access this index location when comparing our underperformers with potential transfer targets.
    
    """
    
    
    # First put our current team ids into a list 
    team_ids = current_team.id.values.tolist()
    
    gw = round_player_set["round"][0]
    
    # Now get these ids from the rd_2_players set
    current_team_next = round_player_set[round_player_set.id.isin(team_ids)].sort_values(by="element_type", 
                                                                                         ascending=True)
    
    ### Now we'll call a function that handles players who have a multiple (double or more) gameweek
    current_team_next = double_or_missing(current_team_next, gw)
    
    # Get the ids for players in our team that are available next week
    cteam_next_ids = current_team_next.id.values.tolist()
    #ut_ids = ut.id.values.tolist()
    
    # Check if there are any players in our team not available for the next gameweek?
    missing_player_ids = np.setdiff1d(team_ids, cteam_next_ids)

    gw = round_player_set["round"][0] # Get the gameweek 

    # If we have unavailable players:
    if len(missing_player_ids) > 0:

        for i, j in enumerate(missing_player_ids): # loop thro the missing players
            player = current_team[current_team.id==j] # Get the players last data point from the last gw
            current_team_next = current_team_next.append(player) # Add this data point to the current team
            current_team_next["round"].iloc[-1]=gw # Set the players gameweek to the current gameweek
            current_team_next.total_points.iloc[-1]=0 # Set the pints to 0 as they did not feature
    
    
    # Sort by points, cost for each position
    team_gk = current_team_next[current_team_next.element_type==1].sort_values(by=["total_points", "cost"], 
                                                                           ascending=[False, True])
    team_def = current_team_next[current_team_next.element_type==2].sort_values(by=["total_points", "cost"], 
                                                                                ascending=[False, True])
    team_mid = current_team_next[current_team_next.element_type==3].sort_values(by=["total_points", "cost"], 
                                                                                ascending=[False, True])
    team_att = current_team_next[current_team_next.element_type==4].sort_values(by=["total_points", "cost"], 
                                                                                ascending=[False, True])
    
    # Join the positions back into a dataframe
    team_sorted_for_transfer = pd.DataFrame(columns = current_team.columns)
    team_sorted_for_transfer = team_sorted_for_transfer.append(team_gk).append(team_def).append(team_mid).append(team_att)
    team_sorted_for_transfer = team_sorted_for_transfer.reset_index()
    team_sorted_for_transfer = team_sorted_for_transfer.drop("index", axis=1)

    

    #Index for worst in each pos, gk=1, def = 6, mid = 11, att = 14
    
    return team_sorted_for_transfer

In [367]:
def net_loss(df, targets, g_idx, d_idx, m_idx, f_idx):
    """
    In this function we calculate the potential loss of points of each player to be considered for transfer into 
    the squad and save them to a list. We then organise loss values in ascending order and create a list of 
    indexes that the ordered values are found in the initial loss list
    df = team_sorted_for_transfer
    """
    
    gk, dfn, mid, fwd = g_idx, d_idx, m_idx, f_idx
    
    # Set an empty list to store our player losses
    net_loss = []
    
    
    ### ADD IN SOME FUNCIONALITY TO CHECK THAT THE DF.ID AND TARGET.ID ARE NOT THE SAME, IE WE'RE NOT TRYING TO 
    ### TRANSFER IN THE SAME PLAYER SON OUT SON IN, KANE OUT KANE IN ETC ETC
    ### THIS IS PRACTICALLY IMPOSSIBLE BUT MAYBE NOT THEORETICALLY SO
    
    ### THE ABOVE COMMENT BEGS THE QUESTION - WHAT HAPPENS IF MY CURRENT TEAM ARE THE BEST SCORING PLAYERS 
    ### IN THE LEAGUE... WE NEED TO MAKE SURE THAT NO TRANSFERS OCCUR IN THAT CASE!!
    
    # calculate the potential loss each transfer would incur
    gk_loss = df.total_points.iloc[1] - targets.total_points.iloc[0]
    def_loss = df.total_points.iloc[6] - targets.total_points.iloc[1]
    mid_loss = df.total_points.iloc[11] - targets.total_points.iloc[2]
    fwd_loss = df.total_points.iloc[14] - targets.total_points.iloc[3]

    # Add each loss to the net_loss list which is our unordered list of net losses
    net_loss.extend([gk_loss, def_loss, mid_loss, fwd_loss])

    # Create an ordered list of net losses high to low
    ordered_loss = net_loss.copy()
    ordered_loss.sort()

    # Now create an ordered list of indexes - i.e the index in net_loss that we can find values in the ordered_loss
    # Note the values have 1 added to equate to the element types 1=gk, 2=def, 3=mid, 4=fwd
    ordered_idx = [net_loss.index(ordered_loss[x])+1 for x in range(len(ordered_loss))]
    
    return ordered_idx

In [368]:
def make_transfers(order_index, team_sorted_for_transfer, team_cost, targets):
    
    """
    In this function we execute the transfers.  We're passed the preferred transfer targets and then we scroll 
    thro from most preferred to least and transfer the first one we can afford in the budget.
    """


    g_idx, d_idx, m_idx, f_idx = 1, 6, 11, 14

    #team_cost = base_team_gw1.cost.sum()/10
    #cost_out = team_sorted_for_transfer.iloc[11].cost/10

    #cost_in = float(targets[targets.element_type==3].cost/10)

    #new_budget = team_cost - cost_out
    #new_team_cost = new_budget + cost_in
    new_team_cost = 0

    best_team = team_sorted_for_transfer.copy()

    for m, n in enumerate(order_index):
        if n == 1: # i.e if we're looking to swap a goalkeeper
            #print(n)
            cost_out = team_sorted_for_transfer.iloc[g_idx].cost/10
            cost_in = float(targets[targets.element_type==1].cost/10)
            new_budget = team_cost - cost_out
            new_team_cost = new_budget + cost_in
            if new_team_cost <= 100:
                best_team = best_team.drop(index=g_idx, axis=0)
                best_team.loc[g_idx] = targets.iloc[0]
                best_team = best_team.sort_index()
                break
            else:
                continue

        elif n == 2: # i.e if we're looking to swap a defender
            #print(n)
            cost_out = team_sorted_for_transfer.iloc[d_idx].cost/10
            cost_in = float(targets[targets.element_type==2].cost/10)
            new_budget = team_cost - cost_out
            new_team_cost = new_budget + cost_in
            if new_team_cost <= 100:
                best_team = best_team.drop(index=d_idx, axis=0)
                best_team.loc[d_idx] = targets.iloc[1]
                best_team = best_team.sort_index()
                break
            else:
                continue

        elif n == 3: # i.e if we're looking to swap a midfielder
            #print(n)
            cost_out = team_sorted_for_transfer.iloc[m_idx].cost/10
            cost_in = float(targets[targets.element_type==3].cost/10)
            new_budget = team_cost - cost_out
            new_team_cost = new_budget + cost_in
            if new_team_cost <= 100:
                best_team = best_team.drop(index=m_idx, axis=0)
                best_team.loc[m_idx] = targets.iloc[2]
                best_team = best_team.sort_index()
                break
            else:
                continue

        if n == 4: # i.e if we're looking to swap an attacker
            #print(n)
            cost_out = team_sorted_for_transfer.iloc[f_idx].cost/10
            cost_in = float(targets[targets.element_type==4].cost/10)
            new_budget = team_cost - cost_out
            new_team_cost = new_budget + cost_in
            if new_team_cost <= 100:
                best_team = best_team.drop(index=f_idx, axis=0)
                best_team.loc[f_idx] = targets.iloc[3]
                best_team = best_team.sort_index()
                break
            else:
                continue
    
    team_cost = best_team.cost.sum()/10
    
    ### POSSIBLE BUG REMAINS - if no transfer target can be transferred in within budget the team remains unchanged
    ### We're relying on one of the 4 targets coming in under budget. It may pay to increase the target pool to 
    ### Two players per position


    # Review our new team            
    return best_team, team_cost

In [369]:
def append_teams(gw1_team, updated_team):
    
    season = gw1_team.append(updated_team)
    
    return season

In [370]:
def generate_teams():
    
    
    # step 1First let's generate the gw1_team, get the set of players to choose from
    round_player_set = round_players(data, 1)
    
    # step 2 Now generate the baseline team
    gw1_team, team_cost = baseline_team(round_player_set)
    
    updated_team = gw1_team.copy()
    appended_teams = gw1_team.copy()
    
    # Create a list of gwks to iterate thro' - note we'll start from gw2
    gwks = list(range(2, data["round"].unique().tolist()[-1]+1))
    
    for i, j in enumerate(gwks):
        # step 3 generate a the set of players from next gameweek, together with a set of targets
        round_player_set = round_players(data, j)

        # step 4, organise our weakest players in to their appointed transfer index
        team_indexed_for_transfer = team_sorted(round_player_set, updated_team)
        
        # Added target step
        targets = target_generator(round_player_set, updated_team)

        # step 5, get the index of preferred transfers
        targets_indexed = net_loss(team_indexed_for_transfer, targets, 1, 6, 11, 14)

        # step 6, make a transfer and update the team cost
        updated_team, team_cost = make_transfers(targets_indexed, team_indexed_for_transfer, team_cost, targets)
        print(updated_team.shape)


        # step 7, append the updated team to the master season list of teams
        appended_teams = append_teams(appended_teams, updated_team)
        print(j, updated_team.web_name.values, updated_team.index.tolist())
        
    
    return appended_teams, updated_team, team_cost

In [371]:
season, ut, costs = generate_teams()

0    Alisson
1      Konsa
2        Son
3       Kane
Name: web_name, dtype: object 0    14
1    15
2    24
3    21
Name: total_points, dtype: object
(15, 9)
2 ['Guaita' 'Pickford' 'Castagne' 'Gabriel' 'James' 'Saïss' 'Digne' 'Klich'
 'Salah' 'Hendrick' 'Willian' 'Son' 'Calvert-Lewin' 'Wilson' 'Vardy'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
0            Meslier
1            Masuaku
2              Bowen
3    Callum Robinson
Name: web_name, dtype: object 0    10
1    12
2    15
3    13
Name: total_points, dtype: object
(15, 9)
3 ['Guaita' 'Pickford' 'Digne' 'Castagne' 'James' 'Saïss' 'Gabriel' 'Salah'
 'Klich' 'Hendrick' 'Willian' 'Bowen' 'Vardy' 'Wilson' 'Calvert-Lewin'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
0    McCarthy
1    Chilwell
2    Grealish
3     Watkins
Name: web_name, dtype: object 0     9
1    18
2    24
3    19
Name: total_points, dtype: object
(15, 9)
4 ['Pickford' 'Guaita' 'Saïss' 'Gabriel' 'Castagne' 'Digne' 'James' 'Salah'
 'Bowen' 'Hendrick' 'Kli

0    Johnstone
1      Bartley
2         Bale
3        Jesus
Name: web_name, dtype: object 0    10
1    15
2    19
3    13
Name: total_points, dtype: object
(15, 9)
26 ['Guaita' 'Pickford' 'Stones' 'Holgate' 'Azpilicueta' 'Dallas' 'Lowton'
 'McTominay' 'Fernandes' 'Foden' 'Bowen' 'Bale' 'Bamford' 'Werner'
 'Wilson'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
0    Meslier
1       Shaw
2     Mahrez
3       Kane
Name: web_name, dtype: object 0     9
1    14
2    18
3    19
Name: total_points, dtype: object
(15, 9)
27 ['Pickford' 'Guaita' 'Azpilicueta' 'Lowton' 'Stones' 'Dallas' 'Holgate'
 'Bale' 'Fernandes' 'McTominay' 'Foden' 'Bowen' 'Bamford' 'Werner' 'Kane'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
0        Meslier
1        Cancelo
2    Milivojevic
3      Iheanacho
Name: web_name, dtype: object 0    11
1    12
2    11
3    17
Name: total_points, dtype: object
(15, 9)
28 ['Guaita' 'Pickford' 'Stones' 'Dallas' 'Azpilicueta' 'Lowton' 'Holgate'
 'McTominay' 'Fernandes' 'Bo

In [372]:
ut

,id,web_name,round,total_points,element_type,short_name,cum_points,value,cost
0,417,Johnstone,33,6,1,WBA,124.0,2.8,45
1,128,Guaita,33,0,1,CRY,110.0,2.3,48
2,92,Lowton,33,8,2,BUR,93.0,2.1,44
3,200,Dallas,33,6,2,LEE,148.0,2.8,52
4,102,Azpilicueta,33,6,2,CHE,105.0,1.8,59
5,165,Holgate,33,5,2,EVE,71.0,1.5,48
6,219,Evans,33,0,2,LEI,94.0,1.7,55
7,481,Pereira,33,10,3,WBA,132.0,2.4,54
8,309,McTominay,33,3,3,MUN,85.0,1.7,49
9,69,Trossard,33,2,3,BHA,115.0,2.0,57


In [ ]:
names = season.groupby("web_name")

In [ ]:
names.get_group("Stones")

In [38]:
ut

,id,web_name,round,total_points,element_type,short_name,cum_points,value,cost
0,128,Guaita,30,4,1,CRY,109.0,2.3,48
1,157,Pickford,30,0,1,EVE,79.0,1.6,48
2,165,Holgate,30,2,2,EVE,60.0,1.2,48
3,200,Dallas,30,2,2,LEE,124.0,2.4,51
4,92,Lowton,30,1,2,BUR,83.0,1.9,44
5,273,Stones,30,0,2,MCI,122.0,2.3,53
6,102,Azpilicueta,30,0,2,CHE,97.0,1.6,59
7,445,Bowen,30,6,3,WHU,110.0,1.9,59
8,69,Trossard,30,2,3,BHA,109.0,1.9,58
9,309,McTominay,30,1,3,MUN,79.0,1.6,49


In [376]:
# Official check that each gamweek has the same squad count
season["round"].value_counts()

33    15
16    15
2     15
3     15
4     15
5     15
6     15
7     15
8     15
9     15
10    15
11    15
12    15
13    15
14    15
15    15
17    15
32    15
18    15
19    15
20    15
21    15
22    15
23    15
24    15
25    15
26    15
27    15
28    15
29    15
30    15
31    15
1     15
Name: round, dtype: int64

In [377]:
# Let's send the season of to a csv file
season.to_csv("season_squads.csv", index=False)

In [378]:
!ls

FFP_initial_investigation.ipynb season_funcs2.ipynb
README.md                       season_squads.csv
Team_Colours.ipynb              shirts
basic_player_info.csv           team_colors.csv
basic_prepared_players.csv      team_season_functions.ipynb
cumulative_points.csv           team_selections_gw1.ipynb
gw1_functions.ipynb             team_selections_season.ipynb
players.csv                     weekly_calls.ipynb


In [379]:
pd.read_csv("season_squads.csv")

,id,web_name,round,total_points,element_type,short_name,cum_points,value,cost
0,128,Guaita,1,10,1,CRY,10.0,0.2,50
1,157,Pickford,1,8,1,EVE,8.0,0.2,50
2,461,Saïss,1,15,2,WOL,15.0,0.3,50
3,494,Gabriel,1,15,2,ARS,15.0,0.3,50
4,123,James,1,14,2,CHE,14.0,0.3,50
...,...,...,...,...,...,...,...,...,...
490,445,Bowen,33,2,3,WHU,129.0,2.1,60
491,284,Foden,33,0,3,MCI,118.0,2.0,60
492,202,Bamford,33,1,4,LEE,165.0,2.5,65
493,233,Iheanacho,33,0,4,LEI,76.0,1.2,61
